* start -> find_sentiment -> conditions (choose; run_diagnosis, positive response) if (run_dia) -> negative_response -> end // and if (positive_res) -> End

* in run_diagnosis we will try to understand or extract these things from our review -> issue_type, tone of customer, urgency -> this response would be json formate or we want structure output 

* and in basis of these three things we wil try to build response from llm


In [5]:
import os
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal
from pydantic import BaseModel, Field
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser

In [6]:
class ResponseState(TypedDict):
  customer_response:str
  sentiment:str
  issue_type:str 
  tone:str 
  urgency:str 
  final_response:str 


In [11]:
## now try to build a fn for finding sentiment node (with validatation output)

llm= HuggingFaceEndpoint(
  repo_id="mistralai/Mistral-7B-Instruct-v0.2",
  task="chat-completions",
  huggingfacehub_api_token= os.getenv("HUGGINGFACEHUB_API_TOKEN")
)

model= ChatHuggingFace(llm=llm)


def find_sentiment(state:ResponseState):

  class SentimentOutput(BaseModel):
    sentiment: Literal["pos", "neg"]= Field(description="Give the sentiment of the given text")


  parser= PydanticOutputParser(pydantic_object=SentimentOutput)


  prompt= PromptTemplate(
    template="Review the given customer response ->{customer_response} \n and give the accordingly sentiment -> {format_instructions} ",
    input_variables=["customer_response"],
    partial_variables= {"format_instructions": parser.get_format_instructions()}
  )

  global model

  chain= prompt|model|parser

  sentiment= chain.invoke(state).sentiment

  return {"sentiment":sentiment}


## now build fn for positive reponse node

def pos_response(state:ResponseState):

  parser= StrOutputParser()

  prompt= PromptTemplate(
    template="Since customer response is positive so please give a Thank you and positive message for this customer for this customer response -> {customer_response}",
    input_variables=['customer_response']
  )

  global model

  chain= prompt|model|parser 

  pos_res= chain.invoke(state)

  return {"final_response": pos_res}


## now if the response from customer is neg then we have to do diagonsis 

def run_diagnosis(state:ResponseState):

  class DiagnosisOutput(BaseModel):
    issue_type: str= Field(description="Review the customer response and try to give the issue which customer is facing.")
    tone: str= Field(description="Review the customer response and try to give the tone of that customer")
    urgency:str= Field(description="Review the customer response and try to give how much urgency customer has.")

  parser= PydanticOutputParser(pydantic_object=DiagnosisOutput)

  prompt= PromptTemplate(
    template="Review the customer response -> {customer_response} \n and give these instructions output accordingly -> {format_instructions}",
    input_variables=["customer_response"],
    partial_variables={"format_instructions":parser.get_format_instructions()}
  )

  global model 

  chain= prompt|model|parser 

  result= chain.invoke(state)

  return result



## try to build that condition fn which will decide , next which node we have to choose

def checkSentiment(state:ResponseState)->Literal["run_diagnosis", "pos_response"]:

  if state["sentiment"]=='pos':
    return "pos_response"
  
  else:
    return "run_diagnosis"
  


 ## now if we got neg response so we will run_diagnosis to get some more information and nw try to build a fn for neg_response (means what should we response finally if we got neg response)

def neg_response(state:ResponseState):

  parser= StrOutputParser()

  prompt= PromptTemplate(
    template="Generate a helping response for the customer if the customer has issue {issue_type} , tone -> {tone}, urgency -> {urgency} \n\n and customer response is -> {customer_response}", 
    input_variables=["issue_type", "tone", "urgency", "customer_response"]
  ) 

  global model

  chain= prompt|model|parser 

  res= chain.invoke(state) 

  return {"final_response":res}
  


In [12]:
## define the graph

graph= StateGraph(ResponseState)


## define all the nodes need

graph.add_node("find_sentiment", find_sentiment)
graph.add_node("positive_response", pos_response)
graph.add_node("run_diagnosis", run_diagnosis)
graph.add_node("negative_response", neg_response)

## now define the edges
graph.add_edge(START, "find_sentiment")

graph.add_conditional_edges("find_sentiment", checkSentiment)
graph.add_edge("pos_response", END)
graph.add_edge("run_diagnosis", "negative_response")
graph.add_edge("negative_response", END)


## compile the graph

workflow= graph.compile()
workflow

ValueError: Found edge starting at unknown node 'pos_response'